In [1]:
import embedding_handling

In [2]:
collection = embedding_handling.get_vector_store_collection()

In [9]:
# from '09-02-23 - Lecture 2 - The Bronze Age.md', chunk 1
# document id: 'e3aa4638-6cd0-4d7e-9c2e-45df31301983'
# chunk id: 'cf9d24c1-a144-426f-b685-fe243d93eea3'
lecture2_chunk1_embeddings = collection.get(
    ids=['cf9d24c1-a144-426f-b685-fe243d93eea3'],
    include=['embeddings']
)['embeddings'][0]

In [3]:
query = "Where did Heinrich Schliemann discover?"
query_embedding = embedding_handling.get_embedding_from_text(query)

In [12]:
# query_result = collection.query([query_embedding], n_results=3)
query_result = collection.query([query_embedding])

Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


In [17]:
type(query_result)

dict

In [16]:
query_result['documents'][0]

['Heinrich Schliemann\ndiscovers Troy in the 1870s - it’s now known to be a real place! Archaeology is having a revolution. Period of romanticism and philhellenism - Greece gaining independence from Ottoman empire.\n\nfound several layers of different rebuildings of Troy\nTroy - very strategic location at mouth of water trade route\n\nvery destructive digging and some looting - dug a giant hole\n\n“The Treasure of Priam” - lots of wealth - gold & copper items\n\nbro liked to lie, was brilliant linguist but generally questionable & problematic\n\nFueled by success of Troy, Schliemann finds Mycenae\n- again digs straight down, into burial sites “Grave Circle A”\n- finds again lots of gold - remember no gold found in Greece, had to have come from somewhere else via war / trade\n- “palace of Agamemnon”\n\nStill scholarship being conducted on these findings with new information, techniques, and technology. DNA analysis\n\nMinoans 1700 - 1500 BCE\n- don’t know what language they spoke or wha

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [49]:
chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [50]:


messages = [
    SystemMessage(
        content="You are a helpful assistant that converts user queries into declarative statements for the purposes of document retrieval using embedding cosine similarity."
    ),
    HumanMessage(
        content=f"Please convert the following user query into a declarative statement for document retrieval purposes. Focus on capturing the main topic and specific inquiry of the user's question. User's Question: {query}"
    ),
]
result = chat(messages)

In [51]:
result

AIMessage(content='Heinrich Schliemann discovered a specific location.')

In [63]:
modified_query_embedding = embedding_handling.get_embedding_from_text(result.content)

In [64]:
modified_query_result = collection.query([modified_query_embedding])

Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


In [54]:
modified_query_result

{'ids': [['b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3',
   'c82fdc35-043c-4b58-8c20-0e97641a25ee',
   'bb0ab99e-debc-43ff-92f5-acca4167e99d',
   '139bea43-a0a8-4a4e-8a4d-6e8d8a7c954e',
   'a8ba826e-e733-41cc-9adf-1661afe1b6b0',
   'f9453227-664b-41bb-9b9e-21fe3d5acfce',
   'ca0dba15-f991-444d-a105-baf3733ea4e6',
   '08b66ca1-419b-48d9-8ca2-0f6987da4737']],
 'distances': [[0.1339134886938973,
   0.19643870203959835,
   0.19666097315644826,
   0.21127677515727616,
   0.2205950616590019,
   0.22761585378318516,
   0.23223610245222803,
   0.24915482461278615]],
 'metadatas': [[{'document_uuid': '729a947b-ab17-4059-9d58-089f9173598c'},
   {'document_uuid': '168cf4f2-1d88-4633-9380-35bf4dac6957'},
   {'document_uuid': '168cf4f2-1d88-4633-9380-35bf4dac6957'},
   {'document_uuid': '729a947b-ab17-4059-9d58-089f9173598c'},
   {'document_uuid': '168cf4f2-1d88-4633-9380-35bf4dac6957'},
   {'document_uuid': '168cf4f2-1d88-4633-9380-35bf4dac6957'},
   {'document_uuid': '729a947b-ab17-4059-9d58-089f9173598

In [55]:
from chromadb import QueryResult

In [65]:
def query_result_to_strings(
        query_result: QueryResult,
        similarity_cutoff: float = 0.2,
        doc_limit: int = 10
    ) -> list[str]:
    """Given a chromadb QueryResult, return a list of strings from the most relevant documents
    that are within the cosine similarity difference of the cutoff.
    """

    results: list[str] = []

    for i, distance in enumerate(query_result['distances'][0]):
        if distance > similarity_cutoff or i > doc_limit - 1:
            break

        results.append(query_result['documents'][0][i])

    return results

In [66]:
def query_result_to_chunk_ids(
        query_result: QueryResult,
        similarity_cutoff: float = 0.2,
        doc_limit: int = 10
    ) -> list[str]:
    """Given a chromadb QueryResult, return a list of chunk ids from the most relevant documents
    that are within the cosine similarity difference of the cutoff.
    """

    results: list[str] = []

    for i, distance in enumerate(query_result['distances'][0]):
        if distance > similarity_cutoff or i > doc_limit - 1:
            break

        results.append(query_result['ids'][0][i])

    return results

In [67]:
docs = query_result_to_strings(modified_query_result)

In [68]:
docs

['Heinrich Schliemann\ndiscovers Troy in the 1870s - it’s now known to be a real place! Archaeology is having a revolution. Period of romanticism and philhellenism - Greece gaining independence from Ottoman empire.\n\nfound several layers of different rebuildings of Troy\nTroy - very strategic location at mouth of water trade route\n\nvery destructive digging and some looting - dug a giant hole\n\n“The Treasure of Priam” - lots of wealth - gold & copper items\n\nbro liked to lie, was brilliant linguist but generally questionable & problematic\n\nFueled by success of Troy, Schliemann finds Mycenae\n- again digs straight down, into burial sites “Grave Circle A”\n- finds again lots of gold - remember no gold found in Greece, had to have come from somewhere else via war / trade\n- “palace of Agamemnon”\n\nStill scholarship being conducted on these findings with new information, techniques, and technology. DNA analysis\n\nMinoans 1700 - 1500 BCE\n- don’t know what language they spoke or wha

In [69]:
chunk_ids = query_result_to_chunk_ids(modified_query_result)

In [70]:
chunk_ids

['b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3',
 'c82fdc35-043c-4b58-8c20-0e97641a25ee',
 'bb0ab99e-debc-43ff-92f5-acca4167e99d']